In [ ]:
## 9월 28일 정리

In [ ]:
# 다변수 수치미분코드

import numpy as np

def numerical_derivative(f,x):
    
    # f : 미분하려고 하는 다변수 함수
    # x : 모든 변수를 포함하고 있는 numpy array(차원상관없음)
    
    delta_x = 1e-4
    derivative_x = np.zeros_like(x)  # 계산된 수치미분 값을 저장하기 위한 변수
    
    # iterator를 이용하여 입력변수 x에 대해 편미분 수행
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        
        idx = it.multi_index   # 현재의 index를 추출 => tuple형태로 리턴
        
        tmp = x[idx]           # 현재 idx의 값을 잠시 보존. delta_x를 이용한 값으로
                               # ndarray를 수정한 후 함수값을 계산해야 하기 때문
                               # 함수값을 계산한 후 원상복구해야 다음 변수에 대한 편미분을
                               # 정상적으로 수행할 수 있다.
        
        x[idx] = tmp + delta_x
        fx_plus_delta = f(x)   # f(x + delta_x)
        
        x[idx] = tmp - delta_x
        fx_minus_delta = f(x)   # f(x - delta_x)
        
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
    
        x[idx] = tmp
        
        it.iternext()
        
    return derivative_x

In [2]:


# Logistic Regression을 python, tensorflow, sklearn으로 각각구현해 보아요!
# 처음은 독립변수가 1개인 걸로 가요!!

import numpy as np
import tensorflow as tf
from sklearn import linear_model

# 수치미분함수를 들고와서 사용해요!
# ##################################
# 다변수 수치미분코드

def numerical_derivative(f,x):
    
    # f : 미분하려고 하는 다변수 함수
    # x : 모든 변수를 포함하고 있는 numpy array(차원상관없음)
    
    delta_x = 1e-4
    derivative_x = np.zeros_like(x)  # 계산된 수치미분 값을 저장하기 위한 변수
    
    # iterator를 이용하여 입력변수 x에 대해 편미분 수행
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        
        idx = it.multi_index   # 현재의 index를 추출 => tuple형태로 리턴
        
        tmp = x[idx]           # 현재 idx의 값을 잠시 보존. delta_x를 이용한 값으로
                               # ndarray를 수정한 후 함수값을 계산해야 하기 때문
                               # 함수값을 계산한 후 원상복구해야 다음 변수에 대한 편미분을
                               # 정상적으로 수행할 수 있다.
        
        x[idx] = tmp + delta_x
        fx_plus_delta = f(x)   # f(x + delta_x)
        
        x[idx] = tmp - delta_x
        fx_minus_delta = f(x)   # f(x - delta_x)
        
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
    
        x[idx] = tmp
        
        it.iternext()
        
    return derivative_x

# ##################################

# Raw Data Loading + Data Preprocessing
# 그런데 이번예제는 이 과정이 필요없죠!!

# Training Data Set
# 지도학습을 하고 있기 때문에 독립변수와 종속변수(label)로 구분해서 데이터를 준비
# 어떤경우에는 이 두개를 아예 분리해서 제공하는 경우도 있어요!
x_data = np.arange(2,21,2).reshape(-1,1)
t_data = np.array([0,0,0,0,0,0,1,1,1,1]).reshape(-1,1)

#########################################################
# python 구현부터 해 보아요!!

# Weight & bias     
W = np.random.rand(1,1)
b = np.random.rand(1)   # (1,)

# 위에서 정의한 W와 b의 값을 구해야 해요!
# 이 값만 구하면 우리의 최종 목적인 model을 완성할 수 있어요!

# loss function(손실함수, cost function , 비용함수)
# 우리 모델의 예측값과 들어온 t_data(정답)
# 입력으로 들어온 x_data와 W,b값을 이용해서 예측값 계산
# t_data(정답)을 비교해되요!!
def loss_func(input_obj):
    
    # input_obj : W와 b를 같이 포함하고 있는 ndarray => [W1 W2 W3 b]
    num_of_bias = b.shape[0]   # num_of_bias : 1
    
    input_W = input_obj[:-1*num_of_bias].reshape(-1,num_of_bias)   # 행렬연산을 하기 위한 W를 생성
    input_b = input_obj[-1*num_of_bias:]                           # bias
    
    
    #  우리 모델의 예측값 : (linear regression model(Wx + b) ==> sigmoid를 적용 )
    z = np.dot(x_data,input_W) + input_b
    y = 1 / ( 1 + np.exp(-1 * z) )  # sigmoid
    
    delta = 1e-7  #  굉장히 작은값을 이용해서 프로그램으로 
                  # 로그 연산시 무한대로 발산하는것을 방지
        
    # cross entropy
    return -np.sum(t_data*np.log(y+delta) + ((1-t_data)*np.log(1-y+delta)))
    
# learning rate
learning_rate = 1e-4

# 학습
for step in range(30000):
    
    input_param = np.concatenate((W.ravel(), b.ravel()),axis=0)   # [W1 W2 W3 b]
    derivative_result = learning_rate* numerical_derivative(loss_func,input_param)

    num_of_bias = b.shape[0] 
    
    W = W - derivative_result[:-1*num_of_bias].reshape(-1,num_of_bias)   # [[W1] [W2] [W3]]
    b = b - derivative_result[-1*num_of_bias:]
    
    
# predict => W,b를 다 구해서!! 우리의 Logistic Regression Model을 완성!!
def logistic_predict(x):  # 공부한 시간이 입력으로 들어와요!!
    
    z = np.dot(x,W) + b
    y = 1 / ( 1 + np.exp(-1*z) )
    
    if y < 0.5:
        result = 0
    else:
        result = 1
        
    return result, y

study_hour = np.array([[13]])
result = logistic_predict(study_hour)
print('####### python 결과값 #########')
print('공부시간 : {}, 결과 : {}'.format(study_hour,result))

####### python 결과값 #########
공부시간 : [[13]], 결과 : (1, array([[0.58065201]]))


In [14]:
### sklearn ### 으로 구현해보아요!

# Logistic Regression Model을 생성해요!
model = linear_model.LogisticRegression()

# Training data set을 이용해서 학습
model.fit(x_data,t_data.ravel())

study_hour = np.array([[13]])
predict_val = model.predict(study_hour)
predict_proba = model.predict_proba(study_hour)
print('####### sklearn 결과값 #########')
print('공부시간 : {}, 결과 : {},{}'.format(study_hour,predict_val,predict_proba))

####### sklearn 결과값 #########
공부시간 : [[13]], 결과 : [0],[[0.50009391 0.49990609]]


In [3]:
# tensorflow를 이용한 구현

# placeholder
X = tf.placeholder(dtype=tf.float32)   # 독립변수가 1개인경우 shape명시하지 않아요! (x_data)
T = tf.placeholder(dtype=tf.float32)   # (t_data)

# Weight & bias
W = tf.Variable(tf.random.normal([1,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# Hypothesis
logit = W * X + b  # matrix 곱연산 하지 않나요?? 
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)


# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    sess.run(train, feed_dict={X:x_data, T:t_data})


study_hour = np.array([13])
result = sess.run(H,feed_dict={X:study_hour})
print('####### tensorflow 결과값 #########')
print('공부시간 : {}, 결과 : {}'.format(study_hour,result))    

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
####### tensorflow 결과값 #########
공부시간 : [13], 결과 : [[0.58152306]]


In [10]:
# Multi Variable Logistic Regression
# 독립변수가 2개 이상인 Logistic Regression

# 학습하는 데이터는 GRE(Graduate Record Examination)와
# GPA(Grade Point Average) 성적 그리고
# Rank(University Rating)에 대한 
# 대학원 합격/ 불합격 정보

# 내 성적 [600,   3.8,   1.] 의 결과 ??!
# 첫번쨰 구현은 sklearn으로 하세요 !



In [38]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy import stats

# Raw Data Loading
df = pd.read_csv('~/notebook_dir/data/admission.csv')

# 결측치 확인
# df.isnull().sum() 결측치가 없다

# 이상치를 확인해서 있으면 제거 !

# fig = plt.figure() # 그림판을 만들어서 여러개의 데이터를 한꺼번에 표현

# fig_admit = fig.add_subplot(1,4,1)
# fig_gre = fig.add_subplot(1,4,2)
# fig_gpa = fig.add_subplot(1,4,3)
# fig_rank = fig.add_subplot(1,4,4)

# fig_admit.boxplot(df['admit'])
# fig_gre.boxplot(df['gre'])
# fig_gpa.boxplot(df['gpa'])
# fig_rank.boxplot(df['rank'])

# fig.tight_layout()
# plt.show()

# 확인했더니 이상치가 있어요 !
# 이상치를 제거해요 !

zscore_threshold = 2.0

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]
    df = df.loc[~df[col].isin(outlier)]
    
    
# print(df.shape)  # (382, 4)

# Training Data set
x_data = df.drop('admit',axis = 1, inplace=False).values # inplace=False : 원본삭제 x
t_data = df['admit'].values.reshape(-1,1)

# 정규화를 진행해야 해요 !!

scaler_x = MinMaxScaler()
scaler_x.fit(x_data)
norm_x_data = scaler_x.transform(x_data) # for python, tensorflow

# sklearn을 이용한 구현
model = linear_model.LogisticRegression()
model.fit(x_data,t_data.ravel())
print('########## sklearn으로 구현한 결과 ##############')
my_score = np.array([[600, 3.8, 1]])
predict_val = model.predict(my_score) # 0 or 1
predict_proba = model.predict_proba(my_score) # (불합격할 확률, 합격할 확률)
print(my_score, predict_val, predict_proba)

# Tensorflow

# placeholder
X = tf.placeholder(shape=[None,3], dtype = tf.float32) # 독립션수의 데이터를 받기 위한 placeholder
T = tf.placeholder(shape=[None,1], dtype = tf.float32) # 종속변수(label)의 데이터를 받기 위한 placeholder

# Weight & bias
W = tf.Variable(tf.random.normal([3,1]), name = 'Weight')
b = tf.Variable(tf.random.normal([1]), name = 'bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-4).minimize(loss)


# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(30000):
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss],
                                         feed_dict={X:norm_x_data, T:t_data})
    if step % 3000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))


my_score = np.array([[600, 3.8, 1]])
scaled_my_score = scaler_x.transform(my_score)
result = sess.run(H,feed_dict={X:scaled_my_score})

print('####### tensorflow 결과값 #########')
print('내 지원정보 : {}, 결과 : {}'.format(my_score,result))    






########## sklearn으로 구현한 결과 ##############
[[600.    3.8   1. ]] [1] [[0.43740782 0.56259218]]
W : [[-0.28196824]
 [ 0.42527026]
 [ 0.5707602 ]], b : [-1.2634385], loss : 0.6515757441520691
W : [[-0.2705611 ]
 [ 0.43524867]
 [ 0.5608847 ]], b : [-1.255288], loss : 0.6502634286880493
W : [[-0.25967857]
 [ 0.44466233]
 [ 0.5506254 ]], b : [-1.248038], loss : 0.6490464806556702
W : [[-0.24926482]
 [ 0.4535667 ]
 [ 0.5400329 ]], b : [-1.2415514], loss : 0.6479062438011169
W : [[-0.2392771 ]
 [ 0.46201462]
 [ 0.5291791 ]], b : [-1.2357485], loss : 0.6468302011489868
W : [[-0.22967282]
 [ 0.4700514 ]
 [ 0.51805687]], b : [-1.230569], loss : 0.6458059549331665
W : [[-0.22041437]
 [ 0.4777162 ]
 [ 0.50673884]], b : [-1.2259473], loss : 0.6448260545730591
W : [[-0.2114698 ]
 [ 0.4850481 ]
 [ 0.49525797]], b : [-1.2218161], loss : 0.6438834071159363
W : [[-0.20280738]
 [ 0.49207747]
 [ 0.4836256 ]], b : [-1.2181075], loss : 0.6429716944694519
W : [[-0.19440031]
 [ 0.49883655]
 [ 0.4718757 ]], b 

In [49]:

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from scipy import stats

# Raw Data Loading
df = pd.read_csv('~/notebook_dir/data/admission.csv')

zscore_threshold = 2.0

for col in df.columns:
    outlier = df[col][np.abs(stats.zscore(df[col])) > zscore_threshold]
    df = df.loc[~df[col].isin(outlier)]
    
    
# Training Data set
x_data = df.drop('admit',axis = 1, inplace=False).values # inplace=False : 원본삭제 x
t_data = df['admit'].values.reshape(-1,1)

# 정규화를 진행해야 해요 !!

scaler_x = MinMaxScaler()
scaler_x.fit(x_data)
norm_x_data = scaler_x.transform(x_data) # for python, tensorflow

# Tensorflow

# placeholder
X = tf.placeholder(shape=[None,3], dtype = tf.float32) # 독립션수의 데이터를 받기 위한 placeholder
T = tf.placeholder(shape=[None,1], dtype = tf.float32) # 종속변수(label)의 데이터를 받기 위한 placeholder

# Weight & bias
W = tf.Variable(tf.random.normal([3,1]), name = 'Weight')
b = tf.Variable(tf.random.normal([1]), name = 'bias')

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# loss function
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=T))

# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-5).minimize(loss)


# session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 학습
for step in range(90000):
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss],
                                         feed_dict={X:norm_x_data, T:t_data})
    if step % 9000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))

# 정확도 측정(Accuracy)

predict = tf.cast(H >= 0.5, dtype = tf.float32) # True -> 1.0 , False -> 0.0
correct = tf.equal(predict,T)
accuracy = tf.reduce_mean(tf.cast(correct, dtype = tf.float32))

accuracy_val = sess.run(accuracy, feed_dict = {X:norm_x_data,T:t_data})
print('Accuracy : {}'.format(accuracy_val))
# prediction
my_score = np.array([[600, 3.8, 1]])
scaled_my_score = scaler_x.transform(my_score)
result = sess.run(H,feed_dict={X:scaled_my_score})

print('####### tensorflow 결과값 #########')
print('내 지원정보 : {}, 결과 : {}'.format(my_score,result))    






W : [[ 0.07212371]
 [ 1.3021982 ]
 [-0.29034412]], b : [0.53358066], loss : 1.0369571447372437
W : [[ 0.05273817]
 [ 1.2807405 ]
 [-0.3120066 ]], b : [0.49496257], loss : 1.0058391094207764
W : [[ 0.03398602]
 [ 1.2597107 ]
 [-0.3330144 ]], b : [0.45751318], loss : 0.9765375852584839
W : [[ 0.01586946]
 [ 1.2393259 ]
 [-0.35339853]], b : [0.42124665], loss : 0.9490499496459961
W : [[-0.0016104 ]
 [ 1.219756  ]
 [-0.37315995]], b : [0.38614428], loss : 0.9233464598655701
W : [[-0.01845615]
 [ 1.2005748 ]
 [-0.39227107]], b : [0.35220915], loss : 0.8992807269096375
W : [[-0.03467251]
 [ 1.1823357 ]
 [-0.4107724 ]], b : [0.31943274], loss : 0.8768923878669739
W : [[-0.05026515]
 [ 1.1645998 ]
 [-0.4286685 ]], b : [0.28779337], loss : 0.8560159802436829
W : [[-0.06524297]
 [ 1.1474336 ]
 [-0.44594553]], b : [0.25727427], loss : 0.8365967273712158
W : [[-0.07961664]
 [ 1.1311095 ]
 [-0.46263066]], b : [0.227861], loss : 0.8186072707176208
Accuracy : 0.3560209572315216
####### tensorflow 결과값